In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import os, json, re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging

/home/dangnth/anaconda3/envs/wordle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load pretrained model and tokenizer

In [2]:
# Config model + quantization
llama3_8b = "meta-llama/Meta-Llama-3-8B-Instruct"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# Load in NF4 to fit 15GB GPU
tokenizer = AutoTokenizer.from_pretrained(llama3_8b, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(llama3_8b, 
                                             quantization_config=nf4_config,
                                             device_map="auto", )

# Set pad token to eos token
tokenizer.pad_token_id = tokenizer.eos_token_id

/home/dangnth/anaconda3/envs/wordle/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|████████████████| 4/4 [00:09<00:00,  2.34s/it]


# Create Wordle game and prompt

In [3]:
from wordle import Wordle
from prompting import Prompt_Helper
game = Wordle()
p = Prompt_Helper()

In [4]:
# Get a few-shot instruction
conversation = p.get_few_shot_instructions(2)
conversation

[{'role': 'system',
  'content': ('Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position, an invalid letter or a correct letter present in the word but in a wrong position. After each guess, we get a result',)},
 {'role': 'system', 'content': 'Example #1:'},
 {'role': 'user', 'content': 'Enter guess #1'},
 {'role': 'assistant', 'content': 'BAGEL'},
 {'role': 'user',
  'content': 'Letter B is not anywhere present in the word, Letter A is not anywhere present in the word, Letter G is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct position, Letter L is not anywhere present in the word, '},
 {'role': 'user', 'content': 'Enter guess #2'},
 {'role': 'assistant', 'content': 'COVER'},
 {'role': 'user',
  'content': 'Letter C is not anywhere present in the word, Letter O is not anywhere present in the word, Letter V is not anywhere present in the word, Letter E at Position 3 is in the word 

# Simple demo

In [ ]:
i = 0

while True:
    input_ids = tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=5,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    guess_str = tokenizer.decode(response, skip_special_tokens=True)
    try:
        guess_eval = game.turn(guess_str)
        user_response = p.explanatory_response(guess_str, guess_eval, i)
        conversation.extend([
            {'role': 'assistant', 'content': guess_str},
            {'role': 'user', 'content': user_response}
        ])
    except ValueError as e:
        conversation.extend([
            {'role': 'assistant', 'content': guess_str},
            {'role': 'user', 'content': str(e)}
        ])
        
    if guess_eval == 'GGGGG':
        conversation.append({'role': 'system', 'content': f"Correct! The answer is {game.get_answer()}"})
    elif i >= 4:
        conversation.append({'role': 'system', 'content': f"5 guesses exhausted! The correct answer is {game.get_answer()}"})
        break
    else:
        i += 1

In [6]:
for turn in conversation:
    print(turn)

{'role': 'system', 'content': ('Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position, an invalid letter or a correct letter present in the word but in a wrong position. After each guess, we get a result',)}
{'role': 'system', 'content': 'Example #1:'}
{'role': 'user', 'content': 'Enter guess #1'}
{'role': 'assistant', 'content': 'BAGEL'}
{'role': 'user', 'content': 'Letter B is not anywhere present in the word, Letter A is not anywhere present in the word, Letter G is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct position, Letter L is not anywhere present in the word, '}
{'role': 'user', 'content': 'Enter guess #2'}
{'role': 'assistant', 'content': 'COVER'}
{'role': 'user', 'content': 'Letter C is not anywhere present in the word, Letter O is not anywhere present in the word, Letter V is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct po